In [43]:
# Import the dependencies.
import pandas as pd
import gmaps
import requests
import matplotlib.pyplot as plt
# Import the API key.
from config import g_key

In [2]:
# 2. Import the WeatherPy_database.csv file from Part 1 as a new DataFrame.
weather_database_df = pd.read_csv("weather_database/WeatherPy_database.csv")
weather_database_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,0,Beira,MZ,-19.84,34.84,55.4,93,20,3.36,few clouds,0.0,0
1,1,Jamestown,US,42.10,-79.24,61.0,82,1,4.70,clear sky,0.0,0
2,2,Esperance,AU,-33.87,121.90,55.4,82,40,19.46,scattered clouds,0.0,0
3,3,Bethel,US,41.37,-73.41,70.0,77,75,3.36,broken clouds,0.0,0
4,4,New Norfolk,AU,-42.78,147.06,50.0,79,8,2.39,clear sky,0.0,0


In [3]:
weather_database_df["Snow (inches)"] = weather_database_df["Snow (inches)"].astype(float)

In [4]:
weather_database_df.dtypes

City_ID                  int64
City                    object
Country                 object
Lat                    float64
Lng                    float64
Max Temp               float64
Humidity                 int64
Cloudiness               int64
Wind Speed             float64
Current Description     object
Rain (inches)          float64
Snow (inches)          float64
dtype: object

In [5]:
# 3a. Prompt the customer for the minimum temperature preference.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 60


In [6]:
# 3b. Prompt the customer for the maximum temperature preference.
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the maximum temperature you would like for your trip? 75


In [7]:
# 3c. Prompt the customer to answer if he or she would like it to be raining or not
rain = str (input("Do you want it to be raining? (yes/no) "))

Do you want it to be raining? (yes/no) no


In [8]:
# 3d. Prompt the customer to answer if he or she would like it to be snowing or not
snow = str (input("Do you want it to be snowing? (yes/no) "))

Do you want it to be snowing? (yes/no) no


In [9]:
# 4a. Have customers narrow their travel searches based on temperature and precipitation.
for index, row in weather_database_df.iterrows():
    if rain == "no":
        rain_inches = weather_database_df["Rain (inches)"] == 0
    else:
        rain_inches = weather_database_df["Rain (inches)"] != 0
    if snow == "no":
        snow_inches = weather_database_df["Snow (inches)"] == 0
    else:
        snow_inches = weather_database_df["Snow (inches)"] != 0
        



In [10]:
# 4b. Have customers narrow their travel searches based on temperature and precipitation.
preferred_cities_df = weather_database_df.loc[(weather_database_df["Max Temp"] <= max_temp) & \
                                          (weather_database_df["Max Temp"] >= min_temp) & 
                                          (weather_database_df["Rain (inches)"] != rain_inches) &
                                          (weather_database_df["Snow (inches)"] != snow_inches)]

preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
1,1,Jamestown,US,42.10,-79.24,61.00,82,1,4.70,clear sky,0.0,0.0
3,3,Bethel,US,41.37,-73.41,70.00,77,75,3.36,broken clouds,0.0,0.0
5,5,Rikitea,PF,-23.12,-134.97,72.79,84,100,15.99,overcast clouds,0.0,0.0
13,13,Guerrero Negro,MX,27.98,-114.06,69.85,77,0,14.32,clear sky,0.0,0.0
14,14,Muzhi,RU,65.37,64.67,65.17,48,30,12.59,scattered clouds,0.0,0.0
21,21,Benguela,AO,-12.58,13.41,66.31,87,28,3.56,scattered clouds,0.0,0.0
31,31,Albany,US,42.60,-73.97,70.00,80,21,1.99,few clouds,0.0,0.0
32,32,Khatanga,RU,71.97,102.50,74.05,38,89,6.58,overcast clouds,0.0,0.0
33,33,Nouadhibou,MR,20.93,-17.03,68.95,88,0,20.71,clear sky,0.0,0.0
34,34,Vila Franca Do Campo,PT,37.72,-25.43,64.40,88,20,5.82,few clouds,0.0,0.0


In [11]:
preferred_cities_df.count()

City_ID                185
City                   185
Country                185
Lat                    185
Lng                    185
Max Temp               185
Humidity               185
Cloudiness             185
Wind Speed             185
Current Description    185
Rain (inches)          185
Snow (inches)          185
dtype: int64

In [12]:
# Create DataFrame called hotel_df to store hotel names along with city, country, and current weather description.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Jamestown,US,61.00,clear sky,42.10,-79.24,
3,Bethel,US,70.00,broken clouds,41.37,-73.41,
5,Rikitea,PF,72.79,overcast clouds,-23.12,-134.97,
13,Guerrero Negro,MX,69.85,clear sky,27.98,-114.06,
14,Muzhi,RU,65.17,scattered clouds,65.37,64.67,
21,Benguela,AO,66.31,scattered clouds,-12.58,13.41,
31,Albany,US,70.00,few clouds,42.60,-73.97,
32,Khatanga,RU,74.05,overcast clouds,71.97,102.50,
33,Nouadhibou,MR,68.95,clear sky,20.93,-17.03,
34,Vila Franca Do Campo,PT,64.40,few clouds,37.72,-25.43,


In [13]:
# Set parameters to search for a hotel.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

In [14]:
# Dependencies and Setup
import requests
import gmaps

# Import API key
from config import g_key

In [37]:
# Iterate through the DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]

    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"

    # Use the search term: "lodging" and our latitude and longitude.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    # Make request and get the JSON data from the search.
    hotels = requests.get(base_url, params=params).json()
    # Grab the first hotel from the results and store the name.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [32]:
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Jamestown,US,61.00,clear sky,42.10,-79.24,Hampton Inn & Suites Jamestown
3,Bethel,US,70.00,broken clouds,41.37,-73.41,Courtyard by Marriott Danbury
5,Rikitea,PF,72.79,overcast clouds,-23.12,-134.97,Pension Maro'i
13,Guerrero Negro,MX,69.85,clear sky,27.98,-114.06,"Casa Laguna, Bed & Breakfast"
14,Muzhi,RU,65.17,scattered clouds,65.37,64.67,
21,Benguela,AO,66.31,scattered clouds,-12.58,13.41,Hotel Praia Morena
31,Albany,US,70.00,few clouds,42.60,-73.97,
32,Khatanga,RU,74.05,overcast clouds,71.97,102.50,"Gostinitsa ""Zapolyar'ye"""
33,Nouadhibou,MR,68.95,clear sky,20.93,-17.03,El Medina
34,Vila Franca Do Campo,PT,64.40,few clouds,37.72,-25.43,Pestana Bahia Praia


In [20]:
# Create map with markers and symbols.
marker_locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure()
markers = gmaps.marker_layer(marker_locations)
fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))

In [21]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
</dl>
"""

In [22]:
# Store the DataFrame Row.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [51]:
# Add a map for the vacation spots and a pop-up marker for each city with Hotel Name, city, country and current description.
marker_locations = hotel_df[["Lat", "Lng"]]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))

In [41]:
# 5. Save and upload the new DataFrame as WeatherPy_vacation.csv.
output_data_file = "weather_database/WeatherPy_vacation.csv"
# Export the City_Data into a CSV.
hotel_df.to_csv(output_data_file, index_label="City_ID")